# Step 1: Installation and Setup

In [ ]:
# Installing TensorFlow
! pip install -q tensorflow-gpu

     |████████████████████████████████| 380.8MB 44kB/s 
     |████████████████████████████████| 3.8MB 45.7MB/s 
     |████████████████████████████████| 450kB 64.3MB/s 
     |████████████████████████████████| 81kB 9.7MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 2.0.2 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 2.0.1 which is incompatible.
ERROR: tensorboard 2.0.2 has requirement grpcio>=1.24.3, but you'll have grpcio 1.15.0 which is incompatible.
ERROR: google-colab 1.0.0 has requirement google-auth~=1.4.0, but you'll have google-auth 1.8.2 which is incompatible.
  Found existing installation: google-auth 1.4.2
    Uninstalling google-auth-1.4.2:
      Successfully uninstalled google-auth-1.4.2
  Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Successfully uninstalled tensorboard-1.15.0
  Found existi

In [ ]:
import tensorflow as tf

In [ ]:
print(tf.__version__)

2.0.0


In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

# Step 2: Importing the dataset from Kaggle

In [ ]:
# install Kaggle API
! pip install -q kaggle

In [ ]:
# create a directory as kaggle
! mkdir -p ~/.kaggle

In [ ]:
# importing kaggle API key to Google Colab
from google.colab import files
uploaded = files.upload()

Saving kaggle.json to kaggle.json


In [ ]:
# copy API key to kaggle directory
! cp kaggle.json ~/.kaggle

In [ ]:
# disable the API key
! chmod 600 /root/.kaggle/kaggle.json

In [ ]:
# import the dataset
! kaggle datasets download -d tongpython/cat-and-dog

 96% 209M/218M [00:04<00:00, 23.9MB/s]
100% 218M/218M [00:04<00:00, 52.2MB/s]


In [ ]:
# unzipping the dataset
! unzip -q /content/cat-and-dog.zip

In [ ]:
training_dir = '/content/training_set/training_set'
test_dir = '/content/test_set/test_set'

# Step 3: Building the model

In [ ]:
# reshape the images
img_shape = (128,128,3)

# Loading the Pre-Trained Model (MobileNetV2)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape = img_shape, include_top=False, weights='imagenet')

9412608/9406464 [==============================] - 1s 0us/step


In [ ]:
base_model.summary()

Model: "mobilenetv2_1.00_128"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
_______________________________________________________________________________

In [ ]:
# freezing the model
base_model.trainable = False

# Defining the custom head for network

In [ ]:
base_model.output

<tf.Tensor 'out_relu/Identity:0' shape=(None, 4, 4, 1280) dtype=float32>

In [ ]:
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()(base_model.output)

In [ ]:
global_average_layer

<tf.Tensor 'global_average_pooling2d/Identity:0' shape=(None, 1280) dtype=float32>

In [ ]:
# output layer / prediction layer
prediction_layer = tf.keras.layers.Dense(units=1, activation='sigmoid')(global_average_layer)

# Define the transfer learning model

In [ ]:
model = tf.keras.models.Model(inputs=base_model.input, outputs=prediction_layer)

In [ ]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 128, 128, 3) 0                                            
__________________________________________________________________________________________________
Conv1_pad (ZeroPadding2D)       (None, 129, 129, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
Conv1 (Conv2D)                  (None, 64, 64, 32)   864         Conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_Conv1 (BatchNormalization)   (None, 64, 64, 32)   128         Conv1[0][0]                      
______________________________________________________________________________________________

In [ ]:
# compile the model
opt = tf.keras.optimizers.RMSprop(learning_rate=0.0001)

In [ ]:
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

# Create Data Generators

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
data_gen_train = ImageDataGenerator(rescale=1/255.0)
data_gen_test = ImageDataGenerator(rescale=1/255.0)

In [ ]:
train_generator = data_gen_train.flow_from_directory(directory=training_dir, target_size=(128,128), batch_size=128, class_mode='binary')

Found 8005 images belonging to 2 classes.


In [ ]:
test_generator = data_gen_test.flow_from_directory(directory=test_dir, target_size=(128,128), batch_size=128, class_mode='binary')

Found 2023 images belonging to 2 classes.


# Step 4: Training the model

In [ ]:
model.fit_generator(generator=train_generator, epochs=5, validation_data=test_generator)

Epoch 1/5
63/63 [==============================] - 39s 620ms/step - loss: 0.6941 - accuracy: 0.5846 - val_loss: 0.5174 - val_accuracy: 0.7514
Epoch 2/5
63/63 [==============================] - 34s 540ms/step - loss: 0.5127 - accuracy: 0.7578 - val_loss: 0.3849 - val_accuracy: 0.8245
Epoch 3/5
63/63 [==============================] - 34s 543ms/step - loss: 0.3959 - accuracy: 0.8455 - val_loss: 0.2959 - val_accuracy: 0.8774
Epoch 4/5
63/63 [==============================] - 34s 542ms/step - loss: 0.3270 - accuracy: 0.8786 - val_loss: 0.2582 - val_accuracy: 0.8908
Epoch 5/5
63/63 [==============================] - 34s 541ms/step - loss: 0.2813 - accuracy: 0.8961 - val_loss: 0.2337 - val_accuracy: 0.8992


# Step 5: Fine Tuning

In [ ]:
base_model.trainable = True

In [ ]:
len(base_model.layers)

155

In [ ]:
fine_tune_at = 100

In [ ]:
# freeze the layers before 100
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable = False

In [ ]:
# compile the model
model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy'])

In [ ]:
# train the model
model.fit_generator(generator=train_generator, epochs=5, validation_data=test_generator)

Epoch 1/5
63/63 [==============================] - 39s 611ms/step - loss: 0.1032 - accuracy: 0.9597 - val_loss: 0.1671 - val_accuracy: 0.9496
Epoch 2/5
63/63 [==============================] - 38s 608ms/step - loss: 0.0201 - accuracy: 0.9948 - val_loss: 0.3456 - val_accuracy: 0.9303
Epoch 3/5
63/63 [==============================] - 38s 604ms/step - loss: 0.0081 - accuracy: 0.9981 - val_loss: 0.1845 - val_accuracy: 0.9590
Epoch 4/5
63/63 [==============================] - 38s 606ms/step - loss: 0.0027 - accuracy: 0.9998 - val_loss: 0.2550 - val_accuracy: 0.9525
Epoch 5/5
63/63 [==============================] - 38s 603ms/step - loss: 0.0019 - accuracy: 0.9996 - val_loss: 0.2300 - val_accuracy: 0.9545
